In [1]:
#### Dependencies
import os
import urllib
from urllib.request import urlopen
import json
import csv
import simplejson
import pandas as pd
import pytest
import base64
import sys
import glob
import time
import progressbar

bar = progressbar.ProgressBar()
for i in bar(range(100)):
    time.sleep(0.02)
    
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
from bs4 import BeautifulSoup
import requests
from pprint import pprint
from datetime import datetime
from ohmysportsfeedspy.v1_0 import API_v1_0
from ohmysportsfeedspy.v1_1 import API_v1_1
from ohmysportsfeedspy.v1_2 import API_v1_2
from ohmysportsfeedspy import MySportsFeeds
msf = MySportsFeeds(version="1.2")

100% (100 of 100) |#######################| Elapsed Time: 0:00:02 Time: 0:00:02


In [2]:
#Web Scrapping
quote_page = "https://www.pro-football-reference.com/super-bowl/"

#Query the website to get HTML page of the url
html_page = urlopen(quote_page)
html_page
#Parse the html using beautiful soup and store in a variable
sb_parser = BeautifulSoup(html_page, "html.parser")
superbowl_id = sb_parser.find(id = "all_super_bowls")
super_bowl_scores = superbowl_id.text.strip() # strip() is used to remove starting and trailing
# super_bowl_scores.to_json("Super_Bowl_Scores")
super_bowl_dumps = json.dumps(super_bowl_scores)
super_bowl_json = json.loads(super_bowl_dumps)
super_bowl_json.replace("\n", " ")
# super_bowl_json.to_json("Super_Bowl_Scores")
# print(super_bowl_json)
sb_DF = StringIO(super_bowl_json)
super_bowl_DF = sb_DF.writable()
#Using Pandas to webscrape
html_df, = pd.read_html(quote_page)
#Store as JSON and CSV
superbowl_DF = html_df.to_csv("Super_Bowl_Scores.csv", sep=",")

In [3]:
# Open CSV

sb_output = "Super_Bowl_Scores.csv"
# sb_open = open(sb_output, "r")
# sb_open_csv = sb_open.read()
# sb_open_csv
sb_open = pd.read_csv(sb_output)
new_col = ["Date", "Winner", "Pts", "Loser", "Pts.1", "MVP", "Stadium", "City", "State"]
del sb_open ["Unnamed: 0"]
sb_open.head()
# sb_open.set_index("SB")
# sb_csv_print = json.loads(sb_open_csv)
# sb_csv_print

,Date,SB,Winner,Pts,Loser,Pts.1,MVP,Stadium,City,State
0,"Feb 4, 2018",LII (52),Philadelphia Eagles,41,New England Patriots,33,Nick Foles,U.S. Bank Stadium,Minneapolis,Minnesota
1,"Feb 5, 2017",LI (51),New England Patriots,34,Atlanta Falcons,28,Tom Brady,NRG Stadium,Houston,Texas
2,"Feb 7, 2016",50,Denver Broncos,24,Carolina Panthers,10,Von Miller,Levi's Stadium,Santa Clara,California
3,"Feb 1, 2015",XLIX (49),New England Patriots,28,Seattle Seahawks,24,Tom Brady,University of Phoenix Stadium,Glendale,Arizona
4,"Feb 2, 2014",XLVIII (48),Seattle Seahawks,43,Denver Broncos,8,Malcolm Smith,MetLife Stadium,East Rutherford,New Jersey


In [312]:
#Open JSON
sb_output = "Super_Bowl_Scores.json"
sb_open = open(sb_output, "r")
sb_open_json = sb_open.read()
sb_json_print = json.loads(sb_open_json)
pprint(sb_json_print)
sb_date_pretty = sb_json_print["Date"]
# pprint(sb_date_pretty.keys())
#Came out much worse than pprint
# e = json.dumps(d, separators=(" ", ":"), sort_keys=True)
# e.replace(":", " ")
# # e.replace("\n", " ")

{'City': {'0': 'Minneapolis',
          '1': 'Houston',
          '10': 'Glendale',
          '11': 'Miami Gardens',
          '12': 'Detroit',
          '13': 'Jacksonville',
          '14': 'Houston',
          '15': 'San Diego',
          '16': 'New Orleans',
          '17': 'Tampa',
          '18': 'Atlanta',
          '19': 'Miami Gardens',
          '2': 'Santa Clara',
          '20': 'San Diego',
          '21': 'New Orleans',
          '22': 'Tempe',
          '23': 'Miami Gardens',
          '24': 'Atlanta',
          '25': 'Pasadena',
          '26': 'Minneapolis',
          '27': 'Tampa',
          '28': 'New Orleans',
          '29': 'Miami Gardens',
          '3': 'Glendale',
          '30': 'San Diego',
          '31': 'Pasadena',
          '32': 'New Orleans',
          '33': 'Palo Alto',
          '34': 'Tampa',
          '35': 'Pasadena',
          '36': 'Pontiac',
          '37': 'New Orleans',
          '38': 'Pasadena',
          '39': 'Miami',
          '4': 'East 

In [313]:
# SB_DF.from_csv("Super_Bowl_Scores.csv")

In [314]:
#SuperBowl DataFrame
sb_dates = []
sb_number = []
sb_mvp = []
sb_winner = []
sb_winPts = []
sb_loser = []
sb_losePts = []
sb_stadium = []
sb_city = []
sb_state = []

for data in sb_json_print:
    sb_dates.append(sb_json_print["Date"])
    sb_number.append(sb_json_print["SB"])
    sb_mvp.append(sb_json_print["MVP"])
    sb_winner.append(sb_json_print["Winner"])
    sb_winPts.append(sb_json_print["Pts"])
    sb_loser.append(sb_json_print["Loser"])
    sb_losePts.append(sb_json_print["Pts.1"])
    sb_stadium.append(sb_json_print["Stadium"])
    sb_city.append(sb_json_print["City"])
    sb_state.append(sb_json_print["State"])
    SB_DF = pd.DataFrame.from_dict(sb_json_print)
SB_DF = SB_DF[["SB", "Date", "Winner", "Pts", "Loser", "Pts.1", "MVP", "Stadium", "City", "State"]]
SB_dataDF = SB_DF.rename(index=str, columns={"Pts": "Winning Score", "Pts.1": "Losing Score"})
SB_dataDF
# sb_date_pretty = sb_json_print["Date"]
# pprint(sb_date_pretty)

,SB,Date,Winner,Winning Score,Loser,Losing Score,MVP,Stadium,City,State
0,LII (52),"Feb 4, 2018",Philadelphia Eagles,41,New England Patriots,33,Nick Foles,U.S. Bank Stadium,Minneapolis,Minnesota
1,LI (51),"Feb 5, 2017",New England Patriots,34,Atlanta Falcons,28,Tom Brady,NRG Stadium,Houston,Texas
10,XLII (42),"Feb 3, 2008",New York Giants,17,New England Patriots,14,Eli Manning,University of Phoenix Stadium,Glendale,Arizona
11,XLI (41),"Feb 4, 2007",Indianapolis Colts,29,Chicago Bears,17,Peyton Manning,Dolphin Stadium,Miami Gardens,Florida
12,XL (40),"Feb 5, 2006",Pittsburgh Steelers,21,Seattle Seahawks,10,Hines Ward,Ford Field,Detroit,Michigan
13,XXXIX (39),"Feb 6, 2005",New England Patriots,24,Philadelphia Eagles,21,Deion Branch,Alltel Stadium,Jacksonville,Florida
14,XXXVIII (38),"Feb 1, 2004",New England Patriots,32,Carolina Panthers,29,Tom Brady,Reliant Stadium,Houston,Texas
15,XXXVII (37),"Jan 26, 2003",Tampa Bay Buccaneers,48,Oakland Raiders,21,Dexter Jackson,Qualcomm Stadium,San Diego,California
16,XXXVI (36),"Feb 3, 2002",New England Patriots,20,St. Louis Rams,17,Tom Brady,Louisiana Superdome,New Orleans,Louisiana
17,XXXV (35),"Jan 28, 2001",Baltimore Ravens,34,New York Giants,7,Ray Lewis+,Raymond James Stadium,Tampa,Florida


In [315]:
NFL_data = "box_scores.csv"
NFL_data = pd.read_csv(NFL_data)
NFL_data.head(5)

,date,visitor,home,visitor_score,home_score,visitor_first_downs,visitor_rushing_first_downs,visitor_passing_first_downs,visitor_penalties,visitor_net_yards,...,home_kick_return_splits,home_int_return_splits,home_penalty_splits,home_fumble_splits,home_field_goals,home_third_down_splits,home_fourth_down_splits,home_total_plays,home_avg_gain,home_time_of_possession
0,"September 7, 2014",Cleveland,Pittsburgh,27,30,23,9,11,3,389,...,2-29,0-0,11-96,1-0,3-3,4-12-33%,1-1-100%,67,7.5,32:27
1,"September 7, 2014",Jacksonville,Philadelphia,17,34,18,2,14,2,306,...,1-24,0-0,6-50,3-2,2-2,8-19-42%,1-1-100%,82,5.1,30:46
2,"September 4, 2014",Green Bay,Seattle,16,36,19,4,13,2,255,...,3-60,1-21,4-69,2-1,2-2,4-11-36%,1-1-100%,66,6.0,33:20
3,"September 7, 2014",Minnesota,St. Louis,34,6,18,6,10,2,355,...,1-26,0-0,13-121,4-0,2-3,4-14-28%,0-0-0%,63,5.0,31:43
4,"September 7, 2014",Cincinnati,Baltimore,23,16,16,4,11,1,380,...,4-109,0-0,3-29,2-1,1-2,8-17-47%,1-2-50%,85,5.0,29:30


In [316]:
team_scores = NFL_data[["date","visitor","home","visitor_score","home_score"]]
team_scores.head(5)

,date,visitor,home,visitor_score,home_score
0,"September 7, 2014",Cleveland,Pittsburgh,27,30
1,"September 7, 2014",Jacksonville,Philadelphia,17,34
2,"September 4, 2014",Green Bay,Seattle,16,36
3,"September 7, 2014",Minnesota,St. Louis,34,6
4,"September 7, 2014",Cincinnati,Baltimore,23,16


In [317]:
team_scores.nlargest(5, "home_score")

,date,visitor,home,visitor_score,home_score
936,"October 23, 2011",Indianapolis,New Orleans,7,62
1190,"October 18, 2009",Tennessee,New England,0,59
1773,"November 18, 2012",Indianapolis,New England,24,59
2558,"December 9, 2012",Arizona,Seattle,0,58
3395,"November 28, 2004",Cleveland,Cincinnati,48,58


In [318]:
team_scores.nlargest(5, "visitor_score")

,date,visitor,home,visitor_score,home_score
744,"October 24, 2010",Oakland,Denver,59,14
1049,"November 15, 2010",Philadelphia,Washington,59,28
3103,"December 15, 2013",Kansas City,Oakland,56,31
3261,"November 18, 2007",New England,Buffalo,56,10
1854,"September 28, 2003",Indianapolis,New Orleans,55,21


In [319]:
NFL_games = "nfl_games.csv"
NFL_games = pd.read_csv(NFL_games)
NFL_date_games = NFL_games.set_index("date")

NFL_date_games = NFL_date_games["2014-09-04":]

In [320]:
NFL_date_games = NFL_date_games[["season","team1","team2","score1","score2"]]
nfl_scores = NFL_date_games
nfl_scores.head()

,season,team1,team2,score1,score2
date,,,,,
2014-09-04,2014,SEA,GB,36,16
2014-09-07,2014,PHI,JAX,34,17
2014-09-07,2014,PIT,CLE,30,27
2014-09-07,2014,BAL,CIN,16,23
2014-09-07,2014,TB,CAR,14,20


In [321]:
team1_scores = nfl_scores.groupby("team1").sum()["score1"]
team2_scores = nfl_scores.groupby("team2").sum()["score2"]

team1_scores_df = pd.DataFrame(team1_scores)
team2_scores_df = pd.DataFrame(team2_scores)

In [322]:
#Scores from 2014 to 2017
nfl_team_scores = pd.concat([team1_scores_df, team2_scores_df], axis=1)
nfl_team_scores["Total Points from 2014 to 2017"] = nfl_team_scores["score1"] + nfl_team_scores["score2"]
del nfl_team_scores["score1"]
del nfl_team_scores["score2"]
nfl_team_scores.nlargest(10, "Total Points from 2014 to 2017")

,Total Points from 2014 to 2017
NE,1631
GB,1482
PIT,1374
ATL,1368
CAR,1342
SEA,1334
NO,1278
ARI,1274
IND,1259
DEN,1250


In [323]:
# Obtained all scores for season, including full season, called DataFrames on all.

nfl_season_2014 = NFL_date_games["2014-09-04":"2015-02-01"]
scores_2014_team1 = nfl_season_2014.groupby("team1").sum()["score1"]
scores_2014_team2 = nfl_season_2014.groupby("team2").sum()["score2"]
scores_2014_df1 = pd.DataFrame(scores_2014_team1)
scores_2014_df2 = pd.DataFrame(scores_2014_team2)
scores_2014 = pd.concat([scores_2014_df1, scores_2014_df2], axis = 1)
scores_2014["Total Points for 2014"] = scores_2014["score1"] + scores_2014["score2"]
del scores_2014["score1"]
del scores_2014["score2"]

nfl_season_2015 = NFL_date_games["2015-09-10":"2016-02-07"]
scores_2015_team1 = nfl_season_2015.groupby("team1").sum()["score1"]
scores_2015_team2 = nfl_season_2015.groupby("team2").sum()["score2"]
scores_2015_df1 = pd.DataFrame(scores_2015_team1)
scores_2015_df2 = pd.DataFrame(scores_2015_team2)
scores_2015 = pd.concat([scores_2015_df1, scores_2015_df2], axis = 1)
scores_2015["Total Points for 2015"] = scores_2015["score1"] + scores_2015["score2"]
del scores_2015["score1"]
del scores_2015["score2"]


nfl_season_2016 = NFL_date_games["2016-09-08":"2017-02-05"]
scores_2016_team1 = nfl_season_2016.groupby("team1").sum()["score1"]
scores_2016_team2 = nfl_season_2016.groupby("team2").sum()["score2"]
scores_2016_df1 = pd.DataFrame(scores_2016_team1)
scores_2016_df2 = pd.DataFrame(scores_2016_team2)
scores_2016 = pd.concat([scores_2016_df1, scores_2016_df2], axis = 1)
scores_2016["Total Points for 2016"] = scores_2016["score1"] + scores_2016["score2"]
del scores_2016["score1"]
del scores_2016["score2"]


nfl_team_scores
scores_2014
scores_2015
scores_2016

,Total Points for 2016
ARI,418
ATL,648
BAL,343
BUF,399
CAR,369
CHI,279
CIN,325
CLE,264
DAL,452
DEN,333


In [326]:
#Getting final row for sums for each DF

nfl_team_scores_sum = nfl_team_scores.loc['Total', 'Sum of Point']= nfl_team_scores['Total Points from 2014 to 2017'].sum()
scores_2014_sum = scores_2014.loc['Total', 'Sum of Point']= scores_2014['Total Points for 2014'].sum()
scores_2015_sum = scores_2015.loc['Total', 'Sum of Point']= scores_2015['Total Points for 2015'].sum()
scores_2016_sum = scores_2016.loc['Total', 'Sum of Point']= scores_2016['Total Points for 2016'].sum()

In [328]:
sum_all = nfl_team_scores["Total Points from 2014 to 2017"].sum()
sum_all

36438.0

# Cumulative Player Stats
### Will Be Merged to Analyze Team Stats

In [79]:
cumStats = glob.glob("Cumulative_Stats/*.csv")

In [80]:
#Merge 8 CSV into 1 combined
# cumStats_combinedCSV= pd.concat([pd.read_csv])
# root = "C:.Users/Michael Diaz/Desktop/GitHub RBDS/Rutgers_Projects/RDSB_Sports/"
# path = os.path.join(root, "Cumulative Stats")
# for path, csv, subdirs, files in os.walk(root):
#     for name in files:
#         print os.path.join(path, name)
# cumStats = glob.glob("Cumulative Stats/*.csv")

#Append all files to have dates

# for file in cumStats:
#     convert_DF = cumStats.insert(0, "Season")
#     convert_DF.insert(2, "Season", len(rows))

#     convert_DF.to_csv(cumStats.csv)
#Loop to Combine ALl DF
cumStats_DF = []
for filename in sorted(cumStats):
    cumStats_DF.append(pd.read_csv(filename))
fullStats_DF = pd.concat(cumStats_DF, join="inner")

fullStats_DF.to_csv("Cumulative_Stats_Combined.csv")
fullStats_DF
# print(cumStats)

,Unnamed: 0,Season,#Player ID,#LastName,#FirstName,#Jersey Num,#Position,#Height,#Weight,#Birth Date,...,#PuntDown,#PuntIn20,#PuntIn20Pct,#PuntTB,#PuntTBPct,#PuntFC,#PuntRet,#PuntRetYds,#PuntRetAvg,#GamesStarted
0,0,2014-2015 Regular,7350,Abdullah,Husain,39.0,DB,"6'0""",204.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,1,2014-2015 Regular,6899,Abdul-Quddus,Isa,42.0,DB,"6'1""",201.0,1989-08-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2,2014-2015 Regular,7861,Aboushi,Oday,75.0,G,"6'5""",315.0,1991-06-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,3,2014-2015 Regular,8062,Acho,Emmanuel,51.0,LB,"6'2""",240.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,4,2014-2015 Regular,6430,Acho,Sam,49.0,OLB,"6'3""",257.0,1988-09-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5,5,2014-2015 Regular,8459,Acosta,Corey,25.0,K,"6'0""",190.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,2014-2015 Regular,6924,Adams,Davante,17.0,WR,"6'1""",215.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
7,7,2014-2015 Regular,7039,Adams,Jeff,70.0,OT,"6'7""",305.0,1989-09-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
8,8,2014-2015 Regular,8134,Adams,Mike,76.0,OT,"6'7""",323.0,1990-03-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
9,9,2014-2015 Regular,7170,Adams,Mike,29.0,SS,"5'11""",205.0,1981-03-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [ ]:
#Merge 8 CSV into 1 combined
# cumStats_combinedCSV= pd.concat([pd.read_csv])
# root = "C:.Users/Michael Diaz/Desktop/GitHub RBDS/Rutgers_Projects/RDSB_Sports/"
# path = os.path.join(root, "Cumulative Stats")
# for path, csv, subdirs, files in os.walk(root):
#     for name in files:
#         print os.path.join(path, name)
cumStats = glob.glob("Cumulative Stats/*.csv")

#Append all files to have dates

# for file in cumStats:
#     convert_DF = cumStats.insert(0, "Season")
#     convert_DF.insert(2, "Season", len(rows))

#     convert_DF.to_csv(cumStats.csv)
#Loop to Combine ALl DF
cumStats_DF = []
for filename in sorted(cumStats):
    cumStats_DF.append(pd.read_csv(filename))
fullStats_DF = pd.concat(cumStats_DF, join="inner")

fullStats_DF.to_csv("Cumulative_Stats_Combined.csv")
fullStats_DF
# print(cumStats)